# Step 1 - Scraping

In [15]:
# Import dependencies
import pandas as pd
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import time

### NASA Mars News
Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. 
Assign the text to variables that you can reference later.

In [16]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

nasa_url = 'https://mars.nasa.gov/news/'

browser.visit(nasa_url)

time.sleep(5)

html = browser.html

soup = BeautifulSoup(html, 'html.parser')

results = soup.find('ul', class_="item_list")

results_title = results.find('div', class_='content_title')

news_title = results_title.a.text

news_link = results.a['href']

news_p = results.find('div', class_='article_teaser_body').text

news_img = results.img['src']

print(f'Title: {news_title}')
print(f'Teaser: {news_link}')
print(f'URL: {news_p}')
print(f'Image link: {news_img}')
print('------')

Title: Q&A with the Student Who Named Ingenuity, NASA's Mars Helicopter
Teaser: /news/8660/qa-with-the-student-who-named-ingenuity-nasas-mars-helicopter/
URL: As a longtime fan of space exploration, Vaneeza Rupani appreciates the creativity and collaboration involved with trying to fly on another planet.
Image link: /assets/overlay-arrow.png
------


In [17]:
# Closes browser window
browser.quit()

### JPL Featured Mars Image
Use splinter to navigate the site and find the image url for the current Featured Mars Image. 
Assign the url string to a variable called featured_image_url.

In [18]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'

browser.visit(jpl_url)

time.sleep(5)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

featured_image_url = (f"https://www.jpl.nasa.gov{soup.find('a', class_='button fancybox')['data-fancybox-href']}")
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16605_ip.jpg'

In [19]:
# Closes browser window
browser.quit()

### Mars Weather Tweet
Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. 
Save the tweet text for the weather report as a variable called mars_weather.

In [20]:
request = requests.get('https://twitter.com/marswxreport?lang=en')

soup = BeautifulSoup(request.content, 'html.parser')

mars_weather = soup.find('p', class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text

mars_weather

'Lovely view of Gale Crater right in the middle of this. https://twitter.com/landru79/status/1255504178719404034\xa0…pic.twitter.com/m38l1OIKHE'

### Mars Facts
Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.

In [21]:
space_facts = 'https://space-facts.com/mars/'

# Use Pandas' `read_html` to parse the url
tables = pd.read_html(space_facts)

df = tables[0]

df.columns = ['Index', 'Value']

df.set_index('Index', inplace=True)

# df
fact_dict = df.to_dict()

fact_dict['Value']

# df.to_html('../Resources/mars_table.html')

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.38 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-87 to -5 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}

### Mars Hemispheres

Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.

In [22]:
mars_pics_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

request = requests.get(mars_pics_url)

soup = BeautifulSoup(request.content, 'html.parser')

results = soup.find_all('a', class_="itemLink product-item")

img_dict = {}

for result in results:
    
    img = (f"https://astrogeology.usgs.gov{result['href']}")
                    
                    
    pic_req = requests.get(img)
    
    
    pic_soup = BeautifulSoup(pic_req.content, 'html.parser')
           
    page_name = (pic_soup.find('h2', class_='title')).text
           
    title = page_name.split(' ')[0]
    
    results = pic_soup.find('li')
    
    full_pic = results.find('a')['href']
           
    img_dict[title] = full_pic
               
img_dict

{'Cerberus': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
 'Schiaparelli': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
 'Syrtis': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
 'Valles': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}

# Step 2 - MongoDB and Flask Application

In [23]:
recent_mars_info={}
recent_mars_info['Latest News'] = {'Title':news_title, 'Teaser': news_p, 'URL': news_link, 'Image': news_img}
recent_mars_info['JPL Featured Image'] = featured_image_url
recent_mars_info['Current Mars Weather'] = mars_weather
recent_mars_info['Hemisphere_images'] = img_dict

df_dict = df.to_dict()

recent_mars_info['Mars Facts'] = df_dict['Value']

recent_mars_info['Mars Facts']

{'Equatorial Diameter:': '6,792 km',
 'Polar Diameter:': '6,752 km',
 'Mass:': '6.39 × 10^23 kg (0.11 Earths)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.38 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Surface Temperature:': '-87 to -5 °C',
 'First Record:': '2nd millennium BC',
 'Recorded By:': 'Egyptian astronomers'}

In [28]:
# client.drop_database(db)

In [30]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.mars_db
collection = db.mars_info

collection.update({}, {"$set": {'mars_info':recent_mars_info}}, upsert=True)


listings = db.mars_info.find()

for listing in listings:
    print(listing)

{'_id': ObjectId('5ea9dc57d07eb8f6b20a91b1'), 'mars_info': {'Latest News': {'Title': "Q&A with the Student Who Named Ingenuity, NASA's Mars Helicopter", 'Teaser': 'As a longtime fan of space exploration, Vaneeza Rupani appreciates the creativity and collaboration involved with trying to fly on another planet.', 'URL': '/news/8660/qa-with-the-student-who-named-ingenuity-nasas-mars-helicopter/', 'Image': '/assets/overlay-arrow.png'}, 'JPL Featured Image': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16605_ip.jpg', 'Current Mars Weather': 'Lovely view of Gale Crater right in the middle of this. https://twitter.com/landru79/status/1255504178719404034\xa0…pic.twitter.com/m38l1OIKHE', 'Hemisphere_images': {'Cerberus': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg', 'Schiaparelli': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg', 'Syrtis': 'http://astropedia.astrogeology.usgs.gov/dow

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  
